analyze He's data to see how much ROH there is.
Turns out there isn't much.
In the two minimally contaminated male samples, there are only about 25cM of ROH genome-wide, I don't think that's enough for reliable inference.
I think the Carribean dataset is a better alternative.

In [2]:
iid="GFW002.003"

In [ ]:
# convert mpileup to hdf5
from toHDF5 import mpileup2hdf5

basepath=f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}"
for ch in range(1, 23):
    print(f'processing chr{ch}')
    mpileup2hdf5(f'{basepath}/mpileup/{iid}.chr{ch}.mpileup', 
        f'/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr{ch}.hdf5',
        iid=iid, outPath=f'{basepath}/hdf5/')


In [1]:
import numpy as np
import sys
sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  # hack to get local package first in path [FROM HARALD - DELETE!!!]
from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind  # Need this import
from hapsburg.PackagesSupport.hapsburg_run import hapsb_chunk_negloglik
from hapsburg.PackagesSupport.hapsburg_run import hapsb_femaleROHcontam
from hapsburg.PackagesSupport.hapsburg_run import hapsb_femaleROHcontam_preload
from hapsburg.PackagesSupport.hapsburg_run import hapsb_multiChunk

In [ ]:
hapsb_ind(iid, chs=range(1,23), 
    path_targets_prefix = f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hdf5",
    h5_path1000g = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    meta_path_ref = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    folder_out=f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/", prefix_out="",
    e_model="readcount", p_model="SardHDF5", post_model="Standard",
    processes=4, delete=False, output=True, save=True, save_fp=False, 
    n_ref=2504, diploid_ref=True, exclude_pops=[], readcounts=True, random_allele=False,
    roh_in=1, roh_out=20, roh_jump=300, e_rate=0.01, e_rate_ref=1e-3, 
    cutoff_post = 0.999, max_gap=0, roh_min_l = 0.04, logfile=True, combine=True, 
    file_result="_roh_full.csv")

In [ ]:
hapsb_chunk_negloglik(iid, 3, 0.05, 0.55, 
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hdf5/{iid}.chr3.hdf5",
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/",
    0.0, roh_in=1, roh_out=20, roh_jump=300, e_rate=0.01, e_rate_ref=0.0,
    save=False, save_fp=False, n_ref=2504, diploid_ref=True, 
    exclude_pops=[], e_model="readcount_contam", p_model="SardHDF5", 
    readcounts=True, random_allele=False, prefix_out="", logfile=False)

In [ ]:
iid="GFW005.A0101"
hapsb_femaleROHcontam(iid, f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/{iid}_roh_full.csv", 
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hdf5/", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    f"/mnt/archgen/users/yilei/Data/He/contamX/{iid}/hapRoh/", processes=4)

In [ ]:
iid="I2978"
contam, se = hapsb_femaleROHcontam(iid, f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/{iid}_roh_full.csv", 
    f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hdf5/", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_auto/maf5_chr", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/", processes=4, e_rate=0.002526)
print(contam)
print(se)
    

In [ ]:
iid="I2978"
contam, se = hapsb_femaleROHcontam_preload(iid, f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/{iid}_roh_full.csv", 
    f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hdf5/", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
    "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
    f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/", processes=4, e_rate=0.002526)
print(contam)
print(se)

In [5]:
print(contam)
print(se)

0.006106721763852304
0.00038893090315670503


In [ ]:
iid="I2978"
chunks = {}
with open(f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/{iid}_roh_full.csv") as f:
    f.readline()
    line = f.readline()
    while line:
        _, _, StartM, EndM, _, lengthM, _, ch, _, _ = line.strip().split(',')
        StartM, EndM, lengthM = float(StartM), float(EndM), float(lengthM)
        if lengthM >= 0.05:
            chunks[ch] = (StartM + 0.005, EndM - 0.005)
        line = f.readline()



logliks = []
for c in np.arange(0, 0.025, 0.001):
    loglik = hapsb_multiChunk(c, chunks, 
        iid, f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hdf5/", 
        "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
        "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
        f"/mnt/archgen/users/yilei/Data/AGDP/contamX/{iid}/hapRoh/", 
        conPop=["CEU"], roh_in=1, roh_out=0, roh_jump=300, e_rate=0.002526, e_rate_ref=1e-3,
        processes=4, save=False, save_fp=False, n_ref=2504, diploid_ref=True, 
        exclude_pops=[], e_model="readcount_contam", p_model="SardHDF5", 
        readcounts=True, random_allele=False, prefix_out="", logfile=False)
    logliks.append(loglik)

import matplotlib.pyplot as plt
plt.plot(np.arange(0, 0.025, 0.001), logliks)


In [ ]:
basepath="/mnt/archgen/users/yilei/Data/AGDP/I4432/cov1over2/batch8/con10"
iid="mixed.con10"
hapsb_ind(iid, chs=range(1,23), 
        path_targets_prefix = f"{basepath}/hdf5",
        h5_path1000g = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr", 
        meta_path_ref = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
        folder_out=f"{basepath}/hapRoh/", prefix_out="",
        e_model="readcount_contam", c=0.1, p_model="SardHDF5", post_model="Standard",
        processes=4, delete=False, output=True, save=True, save_fp=False, 
        n_ref=2504, diploid_ref=True, exclude_pops=[], readcounts=True, random_allele=False,
        roh_in=1, roh_out=20, roh_jump=300, e_rate=0.002, e_rate_ref=1e-3, logfile=True, combine=True, 
        file_result="_roh_full.csv")
    

In [2]:
basepath="/mnt/archgen/users/yilei/Data/AGDP/I4432/cov1over2/batch1/con0"
iid="mixed.con0"
hapsb_ind(iid, chs=range(1,2), 
        path_targets_prefix = f"{basepath}/hdf5_maf5",
        h5_path1000g = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_auto/maf5_chr", 
        meta_path_ref = "/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv",
        folder_out=f"{basepath}/hapRoh_maf5/", prefix_out="",
        e_model="readcount_contam", p_model="SardHDF5", post_model="Standard",
        processes=4, delete=True, output=True, save=True, save_fp=False, 
        n_ref=2504, diploid_ref=True, exclude_pops=[], readcounts=True, random_allele=False,
        c=0.0, roh_min_l_final=0.04, roh_in=1, roh_out=20, roh_jump=300, e_rate=0.002, e_rate_ref=1e-3, 
        logfile=True, combine=True, file_result="_roh_full.csv")

Doing Individual mixed.con0...
Set Output Log path: /mnt/archgen/users/yilei/Data/AGDP/I4432/cov1over2/batch1/con0/hapRoh_maf5/mixed.con0/chr1/hmm_run_log.txt


AssertionError: 

In [4]:
import h5py
import numpy as np
path2hdf5="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_auto/maf5_chr1.hdf5"
f=h5py.File(path2hdf5, 'r')

In [8]:
haps = np.array(f['calldata/GT'])
haps.shape
ref = np.array(f['variants/REF'])
alt = np.array(f["variants/ALT"])

In [6]:
np.max(haps)

12

In [9]:
for site in np.unique(np.where(haps>1)[0]):
    print(np.max(haps[site, :,:]))
    print(f'ref: {ref[site]}, alt: {alt[site]}')

2
ref: A, alt: G
2
ref: C, alt: A
2
ref: G, alt: A
2
ref: G, alt: A
2
ref: C, alt: C
2
ref: G, alt: A
3
ref: T, alt: T
2
ref: G, alt: A
2
ref: T, alt: C
2
ref: T, alt: C
2
ref: A, alt: C
2
ref: C, alt: G
2
ref: G, alt: A
2
ref: C, alt: C
2
ref: C, alt: G
2
ref: G, alt: C
2
ref: C, alt: C
2
ref: G, alt: A
2
ref: A, alt: A
2
ref: T, alt: T
2
ref: G, alt: A
2
ref: G, alt: C
2
ref: C, alt: G
2
ref: C, alt: A
2
ref: T, alt: A
2
ref: C, alt: A
2
ref: G, alt: G
2
ref: T, alt: C
2
ref: T, alt: A
2
ref: G, alt: A
2
ref: C, alt: T
2
ref: G, alt: C
2
ref: C, alt: G
2
ref: G, alt: A
2
ref: G, alt: A
2
ref: C, alt: C
2
ref: G, alt: G
2
ref: C, alt: A
2
ref: C, alt: C
2
ref: A, alt: G
2
ref: G, alt: C
2
ref: G, alt: A
2
ref: G, alt: A
2
ref: A, alt: C
2
ref: G, alt: A
2
ref: C, alt: T
2
ref: G, alt: A
2
ref: G, alt: G
2
ref: A, alt: G
2
ref: G, alt: C
2
ref: C, alt: A
2
ref: T, alt: T
2
ref: G, alt: C
3
ref: C, alt: C
2
ref: A, alt: A
2
ref: A, alt: C
2
ref: C, alt: C
2
ref: C, alt: A
2
ref: T, alt: